In [1]:
from datetime import date
import itertools
import logging
import pandas as pd
import numpy as np
import datetime
import glob
import os
import pickle
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [2]:
logging.getLogger().setLevel(logging.INFO)
# logging.getLogger().setFormat('[%(levelname)s] %(message)s')


def setup_logging(verbose=False):
    try: 
        del logging.root.handlers[:]
    except:
        pass
    if verbose:
        logging.basicConfig(level=logging.INFO, format='[%(levelname)s] %(message)s')
    else:
        logging.basicConfig(level=logging.WARNING, format='[%(levelname)s] %(message)s')
setup_logging(verbose=True)   
logging.info('test')

[INFO] test


In [3]:
fn = 'data/Mike Ashworth NE 2021-06-24 BIOSCAN_Manifest_V1.0.xlsx'
template_fn = '../data/BIOSCAN_Manifest_V1.0_20211207.xlsx'
# spp_fn = '../../analysis/0_partner/data/harbach_spp_201910.txt'

In [4]:
# dump ncbi taxonomy
today = date.today()
taxdump = f'data/taxdump.{today}.tar.gz'
if not os.path.isfile(taxdump):
    ! wget ftp://ftp.ncbi.nlm.nih.gov/pub/taxonomy/taxdump.tar.gz -O {taxdump}

In [5]:
# import ete3
# download and install taxonomy
# ncbi = NCBITaxa()
# only run update if needed
# ncbi.update_taxonomy_database()

In [6]:
def get_data(fn):

    logging.info('reading data from {!r}'.format(fn))
    
    try:
        df = pd.read_excel(fn, dtype=str, index_col=0, keep_default_na=False,
                           sheet_name='TAB1 Specimen Metadata Entry')
    except:
        df = pd.read_excel(fn, dtype=str, index_col=0, keep_default_na=False,
                           sheet_name='Specimen Metadata Entry')
    
    if df.index.duplicated().any():
        logging.error('duplicate SERIES: {}'.format(df.index[df.index.duplicated()].to_list()))
        
    # trailing spaces
    for col in df.columns:
        trailing_spaces = (df[col].str.startswith(' ') | df[col].str.endswith(' '))
        if trailing_spaces.any():
            logging.warning('trailing spaces found in {!r}, removing for validation'.format(col,
                df.loc[trailing_spaces, col].to_list()))
            df[col] = df[col].str.strip()
        
    return df
df = get_data(fn)

[INFO] reading data from 'data/Mike Ashworth NE 2021-06-24 BIOSCAN_Manifest_V1.0.xlsx'


In [7]:
template_df = get_data(template_fn)

[INFO] reading data from '../data/BIOSCAN_Manifest_V1.0_20211207.xlsx'


In [8]:
def check_columns(df, template_df):
    
    logging.info('checking manifest columns against template')
    
    data_cols = set(df.columns)
    template_cols = set(template_df.columns)
        
    if data_cols - template_cols != set():
        logging.warning('extra columns in filled manifest compared to template: {}'.format(data_cols - template_cols))
    if template_cols - data_cols != set():
        logging.error('template columns missing from filled manifest: {}'.format(template_cols - data_cols))
check_columns(df, template_df)

[INFO] checking manifest columns against template


In [9]:
def check_exclude_blanks(df):
    
    logging.info('Checking and excluding blank samples')
    
    # last well of plate expected to be blank
    last_well = df[df['TUBE_OR_WELL_ID'] == 'H12']
    last_well_blanks = (last_well['SCIENTIFIC_NAME'] == 'blank sample')
    if not last_well_blanks.all():
        logging.error('last well H12 is not blank at SERIES {}'.format(
                        last_well[~last_well_blanks].index.to_list()))
    
    is_blank = (df['SCIENTIFIC_NAME'] == 'blank sample')
    blanks = df[is_blank]
    
    logging.info('found {} blank samples based on SCIENTIFIC_NAME'.format(blanks.shape[0]))
    
    # check organism part
    organism_part_pass = (blanks['ORGANISM_PART'] == 'BLANK_SAMPLE')
    if not organism_part_pass.all():
        logging.error('for blanks, ORGANISM_PART expected to be BLANK_SAMPLE, found {}'.format(
                set(blanks.loc[~organism_part_pass, 'ORGANISM_PART'])))
    
    # check that NOT_APPLICABLE is filled in all columns
    blanks_na = blanks.drop(columns=['ORGANISM_PART','SCIENTIFIC_NAME',
                                     'TUBE_OR_WELL_ID','RACK_OR_PLATE_ID',
                                     'PRESERVATIVE_SOLUTION'])
    na_filled = (blanks_na == 'NOT_APPLICABLE').all(axis=0)
    if not na_filled.all():
        logging.warning('for blanks, NOT_APPLICABLE expected, but not found in columns {}'.format(
                            na_filled[~na_filled].index.to_list()))
    # exclude blanks from downstream analysis
    
    df_flt = df[~is_blank]
    
    logging.info('{} samples of {} left for downstream analysis'.format(df_flt.shape[0], df.shape[0]))
    
    return df_flt
        
df = check_exclude_blanks(df)

[INFO] Checking and excluding blank samples
[INFO] found 10 blank samples based on SCIENTIFIC_NAME
[WARNING] for blanks, NOT_APPLICABLE expected, but not found in columns ['CATCH_LOT', 'BOTTLE_DIRECTION', 'WHAT_3_WORDS', 'PHOTOGRAPH_OF_SITE', 'PHOTOGRAPH_OF_CATCH', 'PHOTOGRAPH_OF_PLATE', 'VOUCHER_ID', 'PRESERVATION_APPROACH', 'DATE_OF_PRESERVATION', 'COLLECTOR_SAMPLE_ID', 'ELEVATION', 'OTHER_INFORMATION', 'MISC_METADATA', 'IDENTIFIED_BY', 'IDENTIFIER_AFFILIATION', 'IDENTIFIED_HOW']
[INFO] 950 samples of 960 left for downstream analysis


In [10]:
def get_valid_dict(fn):
    # pick up validation values from data validation sheet
    logging.info('extracting value validation data from {!r}'.format(fn))
    valid_df = pd.read_excel(fn, dtype=str, sheet_name='Data Validation - do not edit')
    valid_dict = dict()
    for col in valid_df.columns:
        valid_dict[col] = valid_df[col].dropna().to_list()
        
    # add 96-well plate well IDs to validation
    row_id = list('ABCDEFGH')
    col_id = range(1,13)
    valid_dict['TUBE_OR_WELL_ID'] = [r + str(c) for (r,c) in itertools.product(row_id, col_id)]
    
    return valid_dict
valid_dict = get_valid_dict(template_fn)

[INFO] extracting value validation data from '../data/BIOSCAN_Manifest_V1.0_20211207.xlsx'


In [11]:
def exclude_missing(series, na_values=None):
    
    # valid missing data 
    no_data = (series.isin(na_values))
    if no_data.sum() > 0:
        logging.info('excluding {} {!r} samples without data in {!r}'.format(no_data.sum(), na_values, series.name))
    return series[~no_data]
    
exclude_missing(df['TIME_OF_COLLECTION'], na_values=['NOT_COLLECTED', ''])

[INFO] excluding 665 ['NOT_COLLECTED', ''] samples without data in 'TIME_OF_COLLECTION'


SERIES
1      10:48:00
2      10:48:00
3      10:48:00
4      10:48:00
5      10:48:00
         ...   
283    10:48:00
284    10:48:00
285    10:48:00
286    10:48:00
287    10:48:00
Name: TIME_OF_COLLECTION, Length: 285, dtype: object

In [12]:
def validate_values(col, df, valid_dict, sep=None, na_values=None, level='e'):
    
    logging.info('validating values in column {!r}'.format(col))
    
    if col not in df.columns:
        logging.error('{!r} column not found in manifest'.format(col))
        return
    if col not in valid_dict.keys():
        logging.error('{!r} column not found in validation sheet'.format(col))
        return
    assert level in ('i','w','e'), '{!r} invalid logging level for validate_values'.format(level)
    
    series = df[col]
    if na_values:
        series = exclude_missing(series, na_values)
    
    col_values = set(series.unique())
    # use separator to split values
    if sep:
        sep_col_values = list()
        for v in col_values:
            sep_col_values.extend([x.strip() for x in v.split(sep)])
        col_values = set(sep_col_values)
    valid_values = set(valid_dict[col])
    invalid_values = col_values - valid_values
    if len(invalid_values) > 0:
        msg = 'invalid values in {!r}: {}'.format(col, invalid_values)
        if level == 'i':
            logging.info(msg)
        elif level == 'w':
            logging.warning(msg)
        elif level == 'e':
            logging.error(msg)
#     else:
#         logging.info('all values valid in {!r}'.format(col))
            
validate_values('ORGANISM_PART', df, valid_dict, sep=" | ")

[INFO] validating values in column 'ORGANISM_PART'
[ERROR] invalid values in 'ORGANISM_PART': {''}


In [13]:
def validate_date(col, df):
    
    logging.info('validating date column {!r}'.format(col))

    if col not in df.columns:
        logging.error('{!r} column not found in manifest'.format(col))
        return
    series = df[col]
    # missing date not allowed
    # series = exclude_missing(series, na_values)
    
    # invalid date formats
    # empty string converted to NaT
    date_series = pd.to_datetime(series, format='%Y-%m-%d', errors='coerce')
    if date_series.isna().any():
        logging.error('invalid dates in {!r}: {}'.format(col, 
                                                         series[date_series.isna()].unique()))
    valid_date_series = date_series[~date_series.isna()]
    
    # dates in future
    future_dates = (valid_date_series > datetime.datetime.today())
    if future_dates.any():
        logging.error('future dates in {!r}: {}'.format(col,
            valid_date_series[future_dates].to_list()))
        
    # dates too old
    old_dates = (valid_date_series < datetime.datetime.strptime('1900-01-01', '%Y-%m-%d'))
    if old_dates.any():
        logging.error("pre-1900 dates in {!r}: {}".format(col,
            valid_date_series[old_dates].to_list())) 
    
    return valid_date_series
df.loc[1,'DATE_OF_COLLECTION'] = 'NOT_COLLECTED'
validate_date('DATE_OF_COLLECTION', df)

[INFO] validating date column 'DATE_OF_COLLECTION'
[ERROR] invalid dates in 'DATE_OF_COLLECTION': ['NOT_COLLECTED' '']


SERIES
2     2021-06-24
3     2021-06-24
4     2021-06-24
5     2021-06-24
6     2021-06-24
         ...    
283   2021-06-24
284   2021-06-24
285   2021-06-24
286   2021-06-24
287   2021-06-24
Name: DATE_OF_COLLECTION, Length: 284, dtype: datetime64[ns]

In [14]:
def validate_time(col, df, na_values=['NOT_COLLECTED']):
    
    logging.info('validating time column {!r}'.format(col))
    
    if col not in df.columns:
        logging.error('{!r} column not found in manifest'.format(col))
        return
    series = df[col]
    series = exclude_missing(series, na_values)
        
    # invalid time formats
    # NB empty string converted to NaT
    time_series = pd.to_datetime(series, format='%H:%M:%S', errors='coerce')
    if time_series.isna().any():
        logging.error('invalid times in {!r}: {}'.format(col, 
                                                         series[time_series.isna()].unique()))
    valid_time_series = time_series[~time_series.isna()]
    
    return valid_time_series
# df.loc[1,'TIME_OF_COLLECTION'] = '23'
validate_time('TIME_OF_COLLECTION', df)

[INFO] validating time column 'TIME_OF_COLLECTION'
[ERROR] invalid times in 'TIME_OF_COLLECTION': ['']


SERIES
1     1900-01-01 10:48:00
2     1900-01-01 10:48:00
3     1900-01-01 10:48:00
4     1900-01-01 10:48:00
5     1900-01-01 10:48:00
              ...        
283   1900-01-01 10:48:00
284   1900-01-01 10:48:00
285   1900-01-01 10:48:00
286   1900-01-01 10:48:00
287   1900-01-01 10:48:00
Name: TIME_OF_COLLECTION, Length: 285, dtype: datetime64[ns]

In [15]:
def validate_time_period(col, df, na_values=['NOT_COLLECTED']):
    
    logging.info('validating time period column {!r}'.format(col))
    
    if col not in df.columns:
        logging.error('{!r} column not found in manifest'.format(col))
        return
    series = df[col]
    series = exclude_missing(series, na_values)

    # conversion with modifications for proper parsing 
    # by pd.Timedelta (does not accept missing data, e.g. 'PT1H')
    # note - will not work for weeks and months
    def convert_iso_duration(s):
        if s == np.nan:
            return np.nan
        if not s.startswith('P') or 'T' not in s:
            return np.nan
        # add days
        if s.startswith('PT'):
            s = s.replace('PT','P0DT')
        # add trailing minutes and seconds
        if s.endswith('H'):
            s += '0M0S'
        elif s.endswith('M'):
            s += '0S'
        try:
            return pd.Timedelta(s)
        except:
            return np.nan
    time_period_series = series.apply(convert_iso_duration)
    if time_period_series.isna().any():
        logging.error('invalid times in {!r}: {}'.format(col, 
            series[time_period_series.isna()].unique()))
    valid_time_period_series = time_period_series[~time_period_series.isna()]
    return valid_time_period_series

# df.loc[1,'DURATION_OF_COLLECTION'] = 'PVT1H'
validate_time_period('DURATION_OF_COLLECTION', df);
# df['DURATION_OF_COLLECTION']

[INFO] validating time period column 'DURATION_OF_COLLECTION'
[ERROR] invalid times in 'DURATION_OF_COLLECTION': ['']


In [16]:
# to be replaced/supported by w3w check
def check_location(df, fn):
    
    logging.info('validating country with coordinates')
    
    loc_col, lat_col, lon_col = 'COLLECTION_LOCATION', 'DECIMAL_LATITUDE', 'DECIMAL_LONGITUDE'

    try:
        loc_df_complete = df[[loc_col, lat_col, lon_col]].copy()
    except:
        logging.error('One of {!r} {!r} {!r} columns not found in manifest'.format(loc_col, lat_col, lon_col))
        return
#     loc_df_isna = (loc_df.isin(na_values)).all(axis=1)
#     if loc_df_isna.any():
#         logging.info('removing {} {!r} samples with missing data from coordinate analysis'.format(
#                 loc_df_isna.sum(), na_values))
#     loc_df_complete = loc_df[~loc_df_isna].copy()
    
    # coordinates in geopy format
    loc_df_complete['coord'] = loc_df_complete.apply(lambda x: '{}, {}'.format(
            x[lat_col], x[lon_col]), axis=1)
    
    # get location data for coordinates
    # use local copy of web query results for re-runs
    # this 
    loc_fn = fn+'_loc.pkl'
    if os.path.isfile(loc_fn):
        locations = pickle.load(open(loc_fn, "rb"))
    else:
        # web map server - openstreetmaps
        logging.info('querying coordinates')
        locator = Nominatim(user_agent='myGeocoder')
        rgeocode = RateLimiter(locator.reverse, min_delay_seconds=1)

        locations = dict()
        for c in loc_df_complete.coord.unique():
            # pre-fill with unknown country
            locations[c] = {'address':{'country':'UNKNOWN'}}
            # check coordniate correctness
            try:
                lat, lon = c.split(', ')
                lat, lon = float(lat), float(lon)
            except:
                logging.error('problem parsing coordinates {!r}'.format(c))
                continue
            if abs(lat) > 90:
                logging.error('invalid latitude {}, should be in [-90,90]'.format(lat))
                continue
            if abs(lon) > 180:
                logging.error('invalid longitude {}, should be in [-180,180]'.format(lon))
                continue
            # web query
            location = rgeocode(c, language='en-gb')
            # rgeocode returns empty location outside of counries and in some other situations
            if location is not None:
                locations[c] = location.raw

        # save locations to file
        pickle.dump(locations, open(loc_fn, "wb"))
        
    # parse country from partner input
    loc_df_complete['partner_country'] = loc_df_complete[loc_col].apply(lambda x: x.split('|')[0].strip().upper())
    
    # extract countries from location data
    loc_countries = dict()
    for coord in locations.keys():
        coord_country = locations[coord]['address']['country'].upper()
        loc_countries[coord] = coord_country
        
        partner_countries = loc_df_complete.loc[loc_df_complete.coord == coord, 'partner_country']
        if partner_countries.nunique() > 1:
            logging.error('multiple partner countries for coordinates {!r}: {}'
                          'skipping coordinate validation'.format(
                                coord, partner_countries.unique()))
            continue
        if partner_countries.shape[0] == 0:
            logging.error('no partner location found for coordinates {!r}'.format(coord))
            continue
        partner_country = partner_countries.iloc[0]
        if coord_country == 'UNKNOWN':
            logging.warning('could not locate country for coordinates {!r}, partner country {!r}'.format(
                    coord, partner_country))
        elif partner_country != coord_country:
            logging.error('country mismatch for coordinates {!r}, partner country {!r}, '
                          'coordinate country {!r}'.format(coord, partner_country, coord_country))
    
    # countries based on coordinates
    loc_df_complete['coord_country'] = loc_df_complete['coord'].replace(loc_countries)
    country_mismatch = (loc_df_complete.coord_country != loc_df_complete.partner_country)

#     if country_mismatch.any():
#         logging.error('coordinates do not match country for SERIES: {}'.format(
#                 country_mismatch[country_mismatch].index.to_list()))
    
    # location data can be re-used, e.g. as an additional field
    return loc_df_complete
# df.loc[2,'DECIMAL_LATITUDE'] = '65'
loc_test = check_location(df, fn)
loc_test

[INFO] validating country with coordinates
[WARNING] could not locate country for coordinates ', ', partner country ''


,COLLECTION_LOCATION,DECIMAL_LATITUDE,DECIMAL_LONGITUDE,coord,partner_country,coord_country
SERIES,,,,,,
1,United Kingdom | England | Shapwick NNR | Roug...,51.161693,-2.8149448,"51.161693, -2.8149448",UNITED KINGDOM,UNITED KINGDOM
2,United Kingdom | England | Shapwick NNR | Roug...,51.161693,-2.8149448,"51.161693, -2.8149448",UNITED KINGDOM,UNITED KINGDOM
3,United Kingdom | England | Shapwick NNR | Roug...,51.161693,-2.8149448,"51.161693, -2.8149448",UNITED KINGDOM,UNITED KINGDOM
4,United Kingdom | England | Shapwick NNR | Roug...,51.161693,-2.8149448,"51.161693, -2.8149448",UNITED KINGDOM,UNITED KINGDOM
5,United Kingdom | England | Shapwick NNR | Roug...,51.161693,-2.8149448,"51.161693, -2.8149448",UNITED KINGDOM,UNITED KINGDOM
...,...,...,...,...,...,...
955,,,,",",,UNKNOWN
956,,,,",",,UNKNOWN
957,,,,",",,UNKNOWN


In [18]:
# l = pd.DataFrame(loc_dict).T
# pd.DataFrame(l['address'].to_list())

In [19]:

# valid_spp = list()
# valid_spp = pd.read_csv(spp_fn, header=None)[0].to_list()
# len(valid_spp)

In [20]:
# to be replaced with ete3 taxonomy lookup
def validate_scientific_names(col, df, spp_fn, na_values = ['NOT_COLLECTED']):
    
    logging.info('validating species names against {!r}'.format(spp_fn))

    # read species names from file
    valid_spp = pd.read_csv(spp_fn, header=None)[0].to_list()
    
    if col not in df.columns:
        logging.error('{!r} column not found in manifest'.format(col))
        return
    series = df[col]
    series = exclude_missing(series, na_values)
    
    correct_genus = series.str.startswith('Anopheles')
    if not correct_genus.all():
        logging.error('expected Anopheles as genus in {!r} column, found: {}'.format(
                      col, series[~correct_genus].unique()))
    
    species = series.str.split(' ').str.get(1)
    correct_species = species.isin(valid_spp)
    
    if not correct_species.all():
        logging.error('species not in {!r} found in {!r} column: {}'.format(
                      spp_fn, col, series[~correct_species].unique()))
# df.loc[1, 'SCIENTIFIC_NAME'] = 'Anapherla askfnvjn'
# validate_scientific_names('SCIENTIFIC_NAME', df, spp_fn)

In [21]:
def check_undeleted_example(df):
    
    logging.info('checking if example row was not deleted')
    
    if (df.index == 'malaise_example-small').any():
        logging.error('example index was not deleted')
    elif (df.RACK_OR_PLATE_ID == 'NBGW-001').any():
        logging.error('example plate ID was not deleted')

In [23]:
def validate(fn, template_fn, verbose=False, version='1.0'):
    '''
    Validation follows the order of columns order in data entry sheet
    '''

    setup_logging(verbose=verbose)

    logging.info('# started validate_partner_manifest_v.{}'.format(version))

    # read data
    df = get_data(fn)
    
    # prepare validation
    template_df = get_data(template_fn)
    check_columns(df, template_df)
    valid_dict = get_valid_dict(template_fn)

    # exclude blanks
    df = check_exclude_blanks(df)
    
    #orange cols
    # BOTTLE_DIRECTION not checked TODO do not allow missing
    validate_values('PRESERVATIVE_SOLUTION', df, valid_dict)
    validate_values('TUBE_OR_WELL_ID', df, valid_dict)
    # CATCH_LOT not checked TODO do not allow missing
    validate_values('BOTTLE_DIRECTION', df, valid_dict)
    validate_values('ORGANISM_PART', df, valid_dict, sep='|')
    validate_values('HAZARD_GROUP', df, valid_dict)
    validate_values('REGULATORY_COMPLIANCE', df, valid_dict)
    validate_date('DATE_OF_COLLECTION', df)
    check_location(df, fn)
    
    # purple cols
    # validate_scientific_names('SCIENTIFIC_NAME', df, spp_fn, na_values=['NOT_COLLECTED'])
    validate_values('LIFESTAGE', df, valid_dict)
    validate_values('SEX', df, valid_dict)
    # HABITAT not checked
    validate_time('TIME_OF_COLLECTION', df)
    validate_time_period('DURATION_OF_COLLECTION', df)
    validate_values('COLLECTION_METHOD', df, valid_dict)
    # DESCRIPTION_OF_COLLECTION_METHOD not checked
    # TODO validate_format('TIME_ELAPSED_FROM_COLLECTION_TO_PRESERVATION', dtype=int)
    # PHOTOGRAPH_* columns not checked
    # VOUCHER_ID not checked
    # PRESERVATION_APPROACH not checked - should match DATE_OF_PRESERVATION
    validate_date('DATE_OF_PRESERVATION', df) # allow for empty values unlike DATE_OF_COLLECTION
    # TODO compare_dates(before='DATE_OF_COLLECTION', after='DATE_OF_PRESERVATION')
    # COLLECTOR_SAMPLE_ID not checked
    # TODO validate_format('ELEVATION', dtype=int)
    # OTHER_INFORMATION	MISC_METADATA	IDENTIFIED_BY	IDENTIFIER_AFFILIATION	IDENTIFIED_HOW not checked
    
    check_undeleted_example(df)
    
    logging.info('# ended validate_partner_manifest_v.{}'.format(version))

    return df

# fn = '../../results/partner_manifests/IRD-Neandersquito_T222Amplicon_Manifest_V2.0.xlsx'
df = validate(fn, template_fn, verbose=True)

[INFO] # started validate_partner_manifest_v.1.0
[INFO] reading data from 'data/Mike Ashworth NE 2021-06-24 BIOSCAN_Manifest_V1.0.xlsx'
[INFO] reading data from '../data/BIOSCAN_Manifest_V1.0_20211207.xlsx'
[INFO] checking manifest columns against template
[INFO] extracting value validation data from '../data/BIOSCAN_Manifest_V1.0_20211207.xlsx'
[INFO] Checking and excluding blank samples
[INFO] found 10 blank samples based on SCIENTIFIC_NAME
[WARNING] for blanks, NOT_APPLICABLE expected, but not found in columns ['CATCH_LOT', 'BOTTLE_DIRECTION', 'WHAT_3_WORDS', 'PHOTOGRAPH_OF_SITE', 'PHOTOGRAPH_OF_CATCH', 'PHOTOGRAPH_OF_PLATE', 'VOUCHER_ID', 'PRESERVATION_APPROACH', 'DATE_OF_PRESERVATION', 'COLLECTOR_SAMPLE_ID', 'ELEVATION', 'OTHER_INFORMATION', 'MISC_METADATA', 'IDENTIFIED_BY', 'IDENTIFIER_AFFILIATION', 'IDENTIFIED_HOW']
[INFO] 950 samples of 960 left for downstream analysis
[INFO] validating values in column 'PRESERVATIVE_SOLUTION'
[ERROR] invalid values in 'PRESERVATIVE_SOLUTION': 